In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!git clone https://github.com/alexeygrigorev/clothing-dataset-small.git

Cloning into 'clothing-dataset-small'...
remote: Enumerating objects: 3839, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (400/400), done.
remote: Total 3839 (delta 9), reused 385 (delta 0), pack-reused 3439 (from 1)
Receiving objects: 100% (3839/3839), 100.58 MiB | 44.07 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [3]:
input_size = 224

train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_ds = train_gen.flow_from_directory(
    "./clothing-dataset-small/train",
    target_size=(input_size, input_size),
    batch_size=32
)

val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

val_ds = val_gen.flow_from_directory(
    "./clothing-dataset-small/validation",
    target_size=(input_size, input_size),
    batch_size=32
)

Found 3068 images belonging to 10 classes.
Found 341 images belonging to 10 classes.


In [4]:
def make_model(input_size=150, learning_rate=0.01, size_inner=100, droprate=0.5):
  base_model = Xception(
      weights='imagenet',
      include_top=False,
      input_shape=(input_size, input_size, 3)
  )
  base_model.trainable = False

  input = keras.Input(shape=(input_size, input_size, 3))

  # base with pretrained conv layers
  base = base_model(input, training=False)
  # vec rep
  vector = keras.layers.GlobalAveragePooling2D()(base)

  # relu layer
  inner = keras.layers.Dense(size_inner, activation="relu")(vector)
  drop = keras.layers.Dropout(droprate)(inner)

  # dense layer/ logit output
  output = keras.layers.Dense(10)(drop)

  model = keras.Model(input, output)

  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  loss = keras.losses.CategoricalCrossentropy(from_logits=True)

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=["accuracy"]
  )

  return model

In [5]:
checkpoint = keras.callbacks.ModelCheckpoint(
    "xception_v4_1_{epoch:02d}_{val_accuracy:.3f}.keras",
    save_best_only=True,
    monitor="val_accuracy",
    mode="max"
)

In [6]:
input_size = 224
learning_rate = 0.0005
size = 100
droprate = 0.2

model = make_model(
    input_size=input_size,
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate
)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 97s 774ms/step - accuracy: 0.5546 - loss: 1.3259 - val_accuracy: 0.7977 - val_loss: 0.6055
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 41s 421ms/step - accuracy: 0.7687 - loss: 0.6526 - val_accuracy: 0.8182 - val_loss: 0.5150
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 40s 414ms/step - accuracy: 0.8198 - loss: 0.5119 - val_accuracy: 0.8006 - val_loss: 0.5334
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 40s 419ms/step - accuracy: 0.8480 - loss: 0.4583 - val_accuracy: 0.8270 - val_loss: 0.4804
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 41s 423ms/step - accuracy: 0.8473 - loss: 0.4434 - val_accuracy: 0.8504 - val_loss: 0.4570
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 40s 417ms/step - accuracy: 0.8735 - loss: 0.3786 - val_accuracy: 0.8358 - val_loss: 0.4662
Epoch 7/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 40s 418ms/step - accuracy: 0.8817 - loss: 0.3411 - val_accuracy: 0.8328 - val_loss: 0.4525
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 40s 415ms/step - accuracy: 0.8850 - loss: 0.3306 - val_accu